In [1]:
import numpy as np
from tqdm import tqdm

In [2]:
with open('input.txt') as fl:
    grid = np.array([ [ int(v) for v in ln.strip() ] for ln in fl.readlines()])

In [3]:
direction_delta = {'N':(-1,0), 'S':(1,0), 'W':(0,-1), 'E':(0,1)}
potential_turns = {'N':('E','W'), 'S':('E','W'), 'E':('N','S'), 'W':('N','S'),
                   'F':('N','S','E','W')}

class Route:
    def __init__(self, pos, target, direction, loss=0, #history=[], 
                 mx=grid.shape[0], my=grid.shape[1]):
        self.pos = pos
        self.target = target
        self.direction = direction
        self.loss = loss
        #self.history = history.copy()
        self.mx, self.my = mx, my
        
    def get_distance(self):
        return abs(self.pos[0] - self.target[0]) + abs(self.pos[1] - self.target[1])
    
    def get_new_routes(self, min_steps=4, max_steps=10):
        new_routes = []
        new_directions = list(potential_turns[self.direction])
        # create the new routes
        for direction in new_directions:
            delta = direction_delta[direction]
            loss = 0
            for i in range(1,max_steps+1):
                new_pos = (self.pos[0] + i*delta[0], self.pos[1] + i*delta[1])
                if 0 <= new_pos[0] < self.mx and 0 <= new_pos[1] < self.my:
                    loss += grid[new_pos[0], new_pos[1]]
                    if i >= min_steps:
                        new_routes.append(Route(new_pos, self.target, direction, self.loss + loss))
        return new_routes    
    
    def __str__(self):
        return 'Route(pos='+str(self.pos)+ ', dir=' + self.direction +\
                ', loss='+str(self.loss) +\
                ', distance=' + str(self.get_distance()) + ')'
    
    def __repr__(self):
        return self.__str__()
                    
            
    

In [4]:
best_routes = {}
best_routes[((0,0),'F')] = Route(pos=(0,0), target=(grid.shape[0]-1, grid.shape[1]-1), direction='F')
improved_pos = [((0,0),'F')]

#min_step, max_step = 1, 3 # part1
min_step, max_step = 4, 10 # part2

# increase the threshold for how far from the target i will continue to consider
pbar = tqdm()
while len(improved_pos) > 0:
    pbar.update()
    new_routes = []
    for pos in improved_pos:
        new_routes.extend(best_routes[pos].get_new_routes(min_step, max_step))
    
    improved_pos = []
    for route in new_routes:
        pos = (route.pos, route.direction)
        if pos not in best_routes:
            best_routes[pos] = route
            improved_pos.append(pos)
        else:
            if route.loss < best_routes[pos].loss:
                best_routes[pos] = route
                improved_pos.append(pos)
    improved_pos = list(set(improved_pos))
            
    min_loss = min(r.loss for r in new_routes)
    
print(min([r.loss for r in best_routes.values() if r.pos == r.target]))

56it [00:17,  8.30it/s]

801
